In [22]:
!pip install PyPDF2


In [23]:
from PyPDF2 import PdfReader

# Path to the uploaded PDF
pdf_path = "Data Structures and Algorithms in Python.pdf"

# Extract text from the PDF
reader = PdfReader(pdf_path)
text = "\n".join([page.extract_text() for page in reader.pages if page.extract_text()])

# Previewing the first 1000 characters to check extraction quality
text[:1000]


'Data Structures and\nAlgorithms in Python\nMichael T. Goodrich\nDepartment of Computer Science\nUniversity of California, Irvine\nRoberto Tamassia\nDepartment of Computer Science\nBrown University\nMichael H. Goldwasser\nDepartment of Mathematics and Computer Science\nSaint Louis University\n\nVP & PUBLISHER Don Fowley\nEXECUTIVE EDITOR Beth Lang Golub\nEDITORIAL PROGRAM ASSISTANT Katherine WillisMARKETING MANAGER Christopher RuelDESIGNER Kenji NgiengSENIOR PRODUCTION MANAGER Janis SooASSOCIATE PRODUCTION MANAGER Joyce Poh\nThis book was set in L aTEX by the authors. Printed and bound by Courier Westford.\nThe cover was printed by Courier Westford.\nThis book is printed on acid free paper. Founded in 1807, John Wiley & Sons, Inc. has been a valued source of knowledge and understanding for \nmore than 200 years, helping people around the world meet their needs and fulﬁ  ll their aspirations. Our company is built on a foundation of principles that include responsibility to the communiti

In [5]:
import re

# Function to clean extracted text
def clean_text(text):
    # Remove extra spaces, new lines, and special characters
    text = re.sub(r'\n+', '\n', text)  # Normalize multiple new lines
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)  # Remove non-ASCII characters
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    return text.strip()

# Clean the extracted text
cleaned_text = clean_text(text)

# Splitting the text into manageable chunks (500 words per chunk for efficient retrieval)
words = cleaned_text.split()
chunk_size = 500  # Approximate word count per chunk
chunks = [" ".join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

# Display the first chunk as a preview
chunks[0]


'Data Structures and Algorithms in Python Michael T. Goodrich Department of Computer Science University of California, Irvine Roberto Tamassia Department of Computer Science Brown University Michael H. Goldwasser Department of Mathematics and Computer Science Saint Louis University VP & PUBLISHER Don Fowley EXECUTIVE EDITOR Beth Lang Golub EDITORIAL PROGRAM ASSISTANT Katherine WillisMARKETING MANAGER Christopher RuelDESIGNER Kenji NgiengSENIOR PRODUCTION MANAGER Janis SooASSOCIATE PRODUCTION MANAGER Joyce Poh This book was set in L aTEX by the authors. Printed and bound by Courier Westford. The cover was printed by Courier Westford. This book is printed on acid free paper. Founded in 1807, John Wiley & Sons, Inc. has been a valued source of knowledge and understanding for more than 200 years, helping people around the world meet their needs and ful ll their aspirations. Our company is built on a foundation of principles that include responsibility to the communities we serve and where 

In [24]:
import PyPDF2

# Open the PDF file
pdf_path = "Data Structures and Algorithms in Python.pdf"

with open(pdf_path, "rb") as file:
    reader = PyPDF2.PdfReader(file)
    text = "\n".join([page.extract_text() for page in reader.pages if page.extract_text()])

# Save extracted text to a file
with open("book_text.txt", "w", encoding="utf-8") as text_file:
    text_file.write(text)

print("✅ PDF text extracted and saved to 'book_text.txt'")


✅ PDF text extracted and saved to 'book_text.txt'


In [17]:
import numpy as np

# Load the extracted text
with open("book_text.txt", "r", encoding="utf-8") as file:
    book_text = file.read()

# Split text into chunks
chunk_size = 500  # Adjust as needed
words = book_text.split()
chunks = [" ".join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

# Save chunks for retrieval
np.save("chunks.npy", np.array(chunks))

print("✅ Text has been split into chunks and saved as 'chunks.npy'")


✅ Text has been split into chunks and saved as 'chunks.npy'


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import joblib

# Load the saved text chunks
chunks = np.load("chunks.npy", allow_pickle=True)

# Create a TF-IDF Vectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)

# Fit and transform the text chunks into TF-IDF vectors
tfidf_matrix = vectorizer.fit_transform(chunks)

# Save the vectorized model and text chunks for retrieval
joblib.dump(vectorizer, "vectorizer.pkl")
np.save("tfidf_matrix.npy", tfidf_matrix.toarray())

print("✅ TF-IDF model is now ready for question answering!")


✅ TF-IDF model is now ready for question answering!


In [1]:
import numpy as np
import joblib
from sklearn.metrics.pairwise import cosine_similarity

# Load the saved TF-IDF model and text chunks
vectorizer = joblib.load("vectorizer.pkl")
tfidf_matrix = np.load("tfidf_matrix.npy")
chunks = np.load("chunks.npy", allow_pickle=True)

print("💬 Type your questions below. Type 'exit' to stop.\n")

while True:
    # Ask a question
    question = input("Ask a question: ")

    # Stop if the user types "exit"
    if question.lower() == "exit":
        print("🚪 Exiting the Q&A system. Have a great day! 😊")
        break

    # Convert the question into a TF-IDF vector
    question_vector = vectorizer.transform([question])

    # Compute cosine similarity between the question and book chunks
    similarities = cosine_similarity(question_vector, tfidf_matrix)

    # Find the most relevant chunk
    best_match_idx = np.argmax(similarities)

    # Retrieve the most relevant text from the book
    best_answer = chunks[best_match_idx]

    print("\n📖 Answer from the book:\n")
    print(best_answer)
    print("\n" + "-"*80 + "\n")


💬 Type your questions below. Type 'exit' to stop.

Ask a question: Explain recursion

📖 Answer from the book:

recursive call may start two others, we call this a binary recursion . •If a recursive call may start three or more others, this is multiple recursion . 4.4.1 Linear Recursion If a recursive function is designed so that each invocation of the body makes at most one new recursive call, this is know as linear recursion . Of the recursions we have seen so far, the implementation of the factorial function (Section 4.1.1) andthegood ﬁbonacci function (Section 4.3) are clear examples of linear recursion. More interestingly, the binary search algorithm (Section 4.1.3) is also an example oflinear recursion , despite the “binary” terminology in the name. The code for binary search (Code Fragment 4.3) includes a case analysis with two branches that lead to recursive calls, but only one of those calls can be reached during a particularexecution of the body. A consequence of the deﬁnition